In [ ]:
import mne
import numpy as np
import random
import warnings
warnings.filterwarnings('ignore')
#mne.set_log_level('WARNING')
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
#%matplotlib inline

import pylab, seaborn as sns
from scipy.stats import ttest_rel, sem

import import_ipynb 
#import nbimporter
from CommonFunctions import loadData

In [ ]:
#File paths
meg_MainFolder = "..\Data\MEG_Data\Data="
figures_MainFolder = "..\Figures\ERFs\Data="

tmin, tmax = -0.8, 0.6

print('tmin = ', tmin)
print('tmax = ', tmax)

dataFolder = meg_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Data folder: ', dataFolder)

figuresFolder = figures_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Figures folder: ', figuresFolder)

random.seed(42)
sensors = 'mag'

In [ ]:
s_id_list = ['13', '16', '17', '18', '21', '23', '26', '28', '29', '30', '31', '32', '33', '34', '35', '36', '38', '39', '40', '41', '42']
s_id_list_corrected = ['23', '26', '30', '31', '34', '35', '38', '39']
print('Number of subjects: ', len(s_id_list))


In [ ]:
filename_ext = ''
if tmin == -0.8 and tmax == 0.6:
    filename_ext = '-elongated'
elif tmin == -0.8 and tmax == 1:
    filename_ext = '-elongated_2'
    
print('filename ext: ', filename_ext)

In [ ]:
conditions = ['Real', 'Real Living', 'Real Object', 'Real - 80%', 'Real - 90%', 'Real - 100%', 
              'Omission', 'Omission Living', 'Omission Object','Omission - 80%', 'Omission - 90%', 
              'Omission - 100%']



In [ ]:
conditions_evoked = [[] for i in range(len(conditions))]

for s_id in s_id_list:
    print('participant: ', s_id)
    if int(s_id) < 23:
        fname = dataFolder+'S'+s_id+'\\'+s_id+'_2_tsss_mc_trans_'+sensors+'_nobase-epochs_afterICA'+filename_ext+'_manually_AR_resampled.fif'
    else: 
        fname = dataFolder+'S'+s_id+'\\block_2_tsss_mc_trans_'+sensors+'_nobase-epochs_afterICA'+filename_ext+'_manually_AR_resampled.fif'

    # Load data
    epochs = loadData(s_id, sensors, fname, resampled=True)
    
    # Separate events 
    # Real Sounds
    real_events = ['living_real_8', 'living_real_9', 'living_real_10',
                  'object_real_8', 'object_real_9', 'object_real_10']
    
    # Real sounds separated by  predictability level
    real_events_8 = ['living_real_8', 'object_real_8']
    real_events_9 = ['living_real_9', 'object_real_9']
    real_events_10 = ['living_real_10', 'object_real_10']
    
    # Real Living sounds
    real_living_events = [e for e in real_events if 'living' in e]
     # Real Object sounds
    real_obj_events = [e for e in real_events if 'object' in e]
    
    # Omissions
    omi_events = [e for e in epochs.event_id if 'omission' in e]
    #Omission separated by  predictability level
    omi_events_8 = [e for e in omi_events if '8' in e]
    omi_events_9 = [e for e in omi_events if '9' in e]
    omi_events_10 = [e for e in omi_events if '10' in e]
    
    # Living omissions
    omi_living_events = [e for e in omi_events if 'living' in e]
    # Object omissions
    omi_obj_events = [e for e in omi_events if 'object' in e]
    
    # List of events separated by predictability or sound identity        
    events_list = [real_events, real_living_events, real_obj_events, real_events_8, real_events_9, 
                   real_events_10, omi_events, omi_living_events, omi_obj_events, omi_events_8, omi_events_9, 
                   omi_events_10]
    
    # Compute evoked data for condition per participant
    for e in range(len(events_list)):
        
        # Check if the participant has given event
        if len(epochs[events_list[e]]) == 0:
            evoked = None
        else:
            tmp_epochs = epochs[events_list[e]].copy()

            #print('omi times: ', tmp_epochs.times)
            if tmin == -0.1 and tmax == 0.6:
                #print('Data shortened from the end!')
                tmp_epochs.crop(tmax=0.5)
            elif tmin == -0.8:
                #print('Warning: Long time scale! Need to crop from baseline too!!')
                tmp_epochs.crop(tmin=-0.4, tmax=0.5)
                #print('Data cropped!')
                
            evoked = tmp_epochs.average()
                
                
        conditions_evoked[e].append(evoked)
      

In [ ]:
print('Number of conditions: ', len(conditions_evoked))
print('Number of participants: ', len(conditions_evoked[0]))

In [ ]:
conditions

In [ ]:
 conditions_evoked[0][0].times

In [ ]:
times = np.arange(-0.4, 0.51, 0.1)

for i in range(len(conditions)):
    evoked = conditions_evoked[i]

    if evoked != None:
        if any(x is None for x in evoked):
            print('none detected!')
            evoked = [x for x in evoked if x != None]
        print('------------ ' + conditions[i] + ' ------------')
        
        # Omissions
        if 'Omi' in conditions[i]:
            grand_avg = mne.grand_average(evoked)
            
            #grand_avg.plot()
            grand_avg_plt_topo = grand_avg.plot_topomap(times=times, vmin=-90, vmax=90)
            topomap_args = dict(vmin=-40, vmax=40)
            ts_args = dict(gfp=True, ylim=dict(mag=[-100, 100]))
            grand_avg_plt = grand_avg.plot_joint(title=conditions[i], times=np.asarray([0, 0.05, 0.1]),
                                                 ts_args=ts_args, topomap_args=topomap_args )
            
            # save plots
            erf_fig_file = figuresFolder + 'ERF_Group_' + str(i) + '_' + sensors + '.png'
            print(erf_fig_file)
            grand_avg_plt.savefig(erf_fig_file)
            erf_fig_file_topo = figuresFolder + 'ERF_Group_' + str(i) + '_' + sensors + '_topo.png'
            grand_avg_plt_topo.savefig(erf_fig_file_topo)
           
        # Real Sounds
        else:
            print('real!!!')
            grand_avg = mne.grand_average(evoked)
            print('times_real: ', times)
            #grand_avg.plot()
            ts_args = dict(gfp=True,  ylim = dict(mag=[-200, 200]))
            topomap_args = dict(vmin=-180, vmax=180)
            grand_avg_plt_topo = grand_avg.plot_topomap(times=times, vmin = -150, vmax=150)
            grand_avg_plt = grand_avg.plot_joint(title=conditions[i], times=np.asarray([0.1, 0.2, 0.3]),
                                                 ts_args=ts_args, topomap_args=topomap_args)
            # save plots
            erf_fig_file = figuresFolder + 'ERF_Group_' + str(i) + '_' + sensors + '.png'
            print(erf_fig_file)
            grand_avg_plt.savefig(erf_fig_file)
            erf_fig_file_topo = figuresFolder + 'ERF_Group_' + str(i) + '_' + sensors + '_topo.png'
            grand_avg_plt_topo.savefig(erf_fig_file_topo)

        plt.show()
    else:
        print('No evoked!!!!!!')
    

#### Plot each condition per participant

In [ ]:
for i in range(len(conditions)):
    print('-----------' + conditions[i] + '----------')
    evoked = conditions_evoked[i]
    
    for j in range(len(s_id_list)):
        print('Participant: ', s_id_list[j])
        evoked[j].plot()
        
    

In [ ]:

conditions_evoked[0][0]